<h1> Differential Diagnosis with Llama 3-8B RAG vs. BioLlama 3 by ContactDoctor

In [ ]:
!pip install -q streamlit langchain_community chromadb huggingface-hub bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 84.3 MB/s eta 0:00:00

In [ ]:
import os
os.makedirs('.streamlit', exist_ok=True)
with open('.streamlit/secrets.toml', 'w') as f:
    f.write("""
[huggingface]
token = "secret_token"

[models]
rag = "meta-llama/Meta-Llama-3-8B-Instruct"
bio = "ContactDoctor/Bio-Medical-Llama-3-8B"
""".lstrip())

In [ ]:
%%writefile app.py

# IMPORT LIBRARY
import streamlit as st
import pandas as pd
import os
import torch

# FOR PARALLELIZATION
from concurrent.futures import ThreadPoolExecutor, as_completed

from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain import PromptTemplate
from sentence_transformers import SentenceTransformer

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import threading
import time
from tenacity import retry, stop_after_attempt, wait_fixed




# CODE BLOCK

PROMPT = """Answer the question based only on the following context,:{context}
Question:{question}
What are the top 10 most likely diagnoses? Be precise, listing one diagnosis per line, and try to cover many unique possibilities.
Ensure the order starts with the most likely. The top 10 diagnoses are."""
MAX_INPUT_TOKENS = 2048 # The sequence length limit of BioMistral-7V
DB_DIR = "./db"

HF_DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
PIPELINE_DEVICE = 0 if torch.cuda.is_available() else -1
HF_TOKEN    = st.secrets["huggingface"]["token"]
model_id    = st.secrets["models"]["rag"]
bio_model_id= st.secrets["models"]["bio"]

### HELPERS TO LOAD THE MODEL ###
#lock to serialize any “move‐model‐on/off GPU” calls
gpu_lock = threading.Lock()

@retry(stop=stop_after_attempt(3), wait=wait_fixed(5))
def hf_from_pretrained(*args, **kwargs):
    return AutoModelForCausalLM.from_pretrained(*args, **kwargs)


### CACHING HEAVY RESOURCES ###

@st.cache_resource(show_spinner=False)
def get_embedding_fn():
  return HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2", model_kwargs = {"device": HF_DEVICE})


bnb_config = BitsAndBytesConfig(
load_in_8bit=True,
bnb_8bit_quant_type="nf4",
llm_int8_threshold=6.0,
llm_int8_enable_fp32_cpu_offload=True,
)



# Load Llama-3-8B RAG
@st.cache_resource(show_spinner=False)
def get_rag_components():
    embed_fn = get_embedding_fn()
    vect  = Chroma(persist_directory=DB_DIR,
                   embedding_function=embed_fn)
    retriever  = vect.as_retriever()

    # If we have CUDA, do 8-bit quantization; otherwise load normally
    kwargs = {
        "use_auth_token": HF_TOKEN,
        "device_map": "auto",
        # torch_dtype only really matters on GPU
        "torch_dtype": torch.bfloat16 if HF_DEVICE=="cuda" else torch.float32,
        "quantization_config": bnb_config,
    }

    if HF_DEVICE == "cuda":
        kwargs["load_in_8bit"] = True

    mod = hf_from_pretrained(model_id, **kwargs)


    pipe  = pipeline(
        "text-generation",
        model=mod,
        tokenizer=AutoTokenizer.from_pretrained(model_id, use_auth_token=HF_TOKEN),
        device=PIPELINE_DEVICE,
        use_fast=True,
        max_length=512,
    )

    prompt = PromptTemplate(template=PROMPT, input_variables=["context", "question"])
    return pipe, retriever, prompt


# Load Bio Model
@st.cache_resource(show_spinner=False)
def get_bio_pipeline():

    # If we have CUDA, do 8-bit quantization; otherwise load normally
    kwargs = {
        "use_auth_token": HF_TOKEN,
        "device_map": "auto",
        # torch_dtype only really matters on GPU
        "torch_dtype": torch.bfloat16 if HF_DEVICE=="cuda" else torch.float32,
        "quantization_config": bnb_config,
    }

    if HF_DEVICE == "cuda":
        kwargs["load_in_8bit"] = True

    bio_mod     = hf_from_pretrained(bio_model_id, **kwargs)

    bio_pipe  =  pipeline(
        "text-generation",
        model=bio_mod,
        tokenizer=AutoTokenizer.from_pretrained(bio_model_id, use_auth_token=HF_TOKEN),
        device=PIPELINE_DEVICE,
        use_fast=True,
        max_length=512,
    )

    return bio_pipe

@st.cache_resource
def get_tokenizer():
    return AutoTokenizer.from_pretrained(model_id, use_auth_token=HF_TOKEN)


# Tricks for Expensive File I/O: streamlit caching
@st.cache_data(show_spinner=False)
def build_vectorstore(uploaded_files):
    os.makedirs("UploadedTextbook", exist_ok=True)
    paths = []
    for f in uploaded_files:
        path = os.path.join("UploadedTextbook", f.name)
        with open(path, "wb") as fp:
            fp.write(f.getbuffer())
        paths.append(path)

    docs = []
    for pdf in paths:
        docs.extend(PyPDFLoader(pdf).load())
    splitter = CharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=1500, chunk_overlap=200
    )
    splits = splitter.split_documents(docs)

    vs = Chroma.from_documents(
        documents=splits,
        embedding_function=get_embedding_fn(),
        persist_directory=DB_DIR,
    )
    vs.persist()
    return True


### HELPERS TO PROCESS THE OUTPUT ###
# Make sure the token inputs are within the limit
def check_length(text):
    tok = get_tokenizer()
    token_count = len(tok.encode(text))
    if token_count > MAX_INPUT_TOKENS:
        st.warning(f"Your input is {token_count} tokens, over the {MAX_INPUT_TOKENS}-token limit. Please shorten it.")
        return False
    return True

def safe_invoke(model_or_chain, *args, **kwargs):
    try:
        if hasattr(model_or_chain, "invoke"):
            return model_or_chain.invoke(*args, **kwargs)
        return model_or_chain(*args, **kwargs)

    except Exception as e:
        st.error(f"MODEL ERROR: {e}")
        return None


def process_case(txt):
    # 1) Retrieve context
    try:
      rag_pipe, rag_retriever, prompt = get_rag_components()
      docs    = rag_retriever.get_relevant_documents(txt)
      context = "\n\n".join(d.page_content for d in docs)

      # 2) Generate with RAG‐LLM
      prompt_text = prompt.format_prompt(context=context, question=txt).to_string()
      rag_out_raw = safe_invoke(rag_pipe, prompt_text, max_new_tokens=128)
      if rag_out_raw:
          raw = rag_out_raw[0]["generated_text"]
          # strip prompt echo
          rag_out = raw[len(prompt_text):].lstrip() if raw.startswith(prompt_text) else raw
      else:
          rag_out = "Error"

      # 3) Generate with Bio‐LLM
      bio_pipe    = get_bio_pipeline()
      bio_prompt  = PROMPT.format(context="", question=txt)
      bio_raw = safe_invoke(bio_pipe, bio_prompt, max_new_tokens=128)
      if bio_raw:
          raw_b = bio_raw[0]["generated_text"]
          bio_out = raw_b[len(bio_prompt):].lstrip() if raw_b.startswith(bio_prompt) else raw_b
      else:
          bio_out = "Error"

      return {"Case": txt, "Llama3+RAG": rag_out, "BioLlama3": bio_out}

    except Exception as e:
        return {"Case": txt, "Llama3+RAG": f"ERROR: {e}", "BioLlama3": ""}

# Then when generating, temporarily move to GPU:
def generate_bio(bio_pipe, prompt):
  with gpu_lock:
    # move to GPU
    bio_pipe.model.to("cuda")
    out = bio_pipe(prompt)[0]["generated_text"]
    # move back to CPU to free VRAM
    bio_pipe.model.to("cpu")
  return out



### STREAMLIT UI ###
st.title("Differential Diagnosis: Llama 3-8B RAG vs. BioLlama 3 by ContactDoctor")
st.caption("Predefined Medical Guidelines for RAG: Kumar & Clark's Clinical Medicine 10th Ed. 2020")

# Additional files
with st.sidebar:
    st.header("Upload additional resources for RAG (type:.pdf)")
    UploadedFiles = st.file_uploader("Upload here and click on 'Upload'", type="pdf", accept_multiple_files=True)

    if st.button("Build Index"):
        try:
          run_id = str(time.time())
          if not UploadedFiles:
              st.error("Select at least one PDF first.")
          else:
              st.session_state["building_index"] = True

          if st.session_state.get("building_index"):
            with st.spinner("Indexing…"):
                build_vectorstore(UploadedFiles, run_id)
            st.success("RAG index is ready!")
            st.session_state["building_index"] = False

        except:
            print("File already exists")

    st.markdown("---")
    st.header("Batch processing case upload (type:.csv)")
    csv_file = st.file_uploader(
        "Upload CSV",
        type="csv",
        accept_multiple_files=False)

### SINGLE CASE ###
st.subheader("SINGLE CASE")
question = st.text_area("Case Narrative:",
                        height=180,
                        placeholder="For example: 22-year-old patient with TB was admitted to hospital today. The patient has been to a country outside Sweden. The patient came back to Sweden from the other country. The patient has had a fever for two weeks and is admitted. The doctor has prescribed a medicine. ")
st.write(f"The number of characters are {len(question)} characters.")

if st.button('Start Processing'):
  if check_length(question):
    with st.spinner("Processing..."):

      tabs = st.tabs(["BIOMode", "RAGMode"])

      with tabs[0]:
        #BioLlama-3 execution
        bio_pipe = get_bio_pipeline()
        bio_prompt = PROMPT.format(context="",question=question)
        raw_bio    = bio_pipe(bio_prompt)[0]["generated_text"]

        #remove the prompt echo
        if raw_bio.startswith(bio_prompt):
            bio_output = raw_bio[len(bio_prompt):].lstrip()
        else:
            bio_output = raw_bio

        st.markdown("**BioLlama 3**")
        st.text(bio_output)

        #free up memory for RAG
        del bio_pipe
        torch.cuda.empty_cache()

      with tabs[1]:
        if not os.path.isdir(DB_DIR) or not os.listdir(DB_DIR):
          st.error("Please upload and build your PDF index first!")
          st.stop()

        rag_pipe, rag_retriever, prompt = get_rag_components()
        docs = rag_retriever.get_relevant_documents(question)
        context = "\n\n".join([d.page_content for d in docs])
        prompt_text = prompt.format_prompt(context=context, question=question).to_string()

        st.markdown("**Llama 3-8B + RAG**")
        raw = rag_pipe(prompt_text)[0]["generated_text"]

        #remove the prompt echo
        if raw.startswith(prompt_text):
            answer = raw[len(prompt_text):].lstrip()
        else:
            answer = raw
        st.text(answer)

        #free up memory from RAG
        del rag_pipe, rag_retriever, prompt
        torch.cuda.empty_cache()

  else:
    st.stop()

### BATCH PROCESSING ###
st.markdown("---")
st.subheader("BATCH MODE")

if csv_file:
    df = pd.read_csv(csv_file)
    if st.button("Start Batch Processing"):
        results = []
        futures = []
        prog = st.progress(0)

        BATCH_WORKERS = 1 if HF_DEVICE != "cuda" else 2
        with ThreadPoolExecutor(max_workers=BATCH_WORKERS) as exe:
            for txt in df["Case"]:
                futures.append(exe.submit(process_case, txt))

            # as each case completes, update progress
            for i, fut in enumerate(as_completed(futures)):
                results.append(fut.result())
                prog.progress((i + 1) / len(futures))

        out_df = pd.DataFrame(results)
        st.download_button(
            "Download Results as CSV",
            data=out_df.to_csv(index=False),
            file_name="ddx_comparison.csv"
        )

<h2>Install local-tunnel </h2>

In [ ]:
!npm install localtunnel

<h2> Run Streamlit in background </h2>

In [ ]:
# AND Expose to the port 8501
!streamlit run /content/app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com